#Introduction

In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block.

Data preparation: Download the points.csv from the data folder.

First, let's use ArcToolbox tools to convert the csv to a feature class (10 pnts)
In ArcGIS Pro, search for "XYTable to Point"
In the dialog, put the "point.csv" from this folder as the input table
Name the output something like "point_fromxy"
Before, you run the model, in the Run button, click the small triangle and select "Copy Python Command"
Paste your python code below in the code block
Run the code block to make sure it works
Compare the python code and the ArcGIS Pro Geoprocessing pane dialog interface

In [1]:

# To get the 10 points 
# Paste the python code from the above instruction here
# make sure you start with import arcpy

import arcpy
arcpy.management.XYTableToPoint(
    in_table=r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\CSV_Data\points.csv",
    out_feature_class=r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\Output_points\point_fromxy.shp",
    x_field="X",
    y_field="Y",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'D:\\LSU_Spring 2025\\GEOG 4057\\Class Projects\\Assignment 6\\Output_points\\point_fromxy.shp'>

Commentating the example code:(20 pnts)
Using InsertCursor to do the same thing but project the geometry to a different spatial reference

The above code only imports the csv to a feature class but not converting from a GCS to a project coordinate system

We will use the following code to do the work and project the data to "NAD 1983 USGS Contiguous USA Albers" system with the WKID = 102039

Look the code up from here: https://epsg.io/102039

In [2]:
import arcpy
import csv
import os

# Edit the path names for your case
csv_file_path = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\CSV_Data\points.csv"
geodatabase_path = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\GIS\Assignment_6\Assignment_6.gdb"
feature_class_name = "point_albers"

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create SpatialReference objects: 
sr_gcs = arcpy.SpatialReference(4269) 
sr_albsers = arcpy.SpatialReference(102039)
#delete if the feature class exists

fcName = os.path.join(geodatabase_path,feature_class_name)

if arcpy.Exists(fcName):
    arcpy.management.Delete(fcName)


# Create a new point feature class

arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr_albsers)
# Add fields to the feature class based on the CSV header

# first, read the exisint fields 
describe = arcpy.da.Describe(fcName)
field_names = [field.name for field in describe["fields"]]


for field in header:
    if field not in field_names:
        arcpy.AddField_management(fcName, field, "TEXT")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@"] + header
with arcpy.da.InsertCursor(fcName, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for table_value in csv_reader:
            x = float(table_value[0]) # note that the first two fields in the csv must be longitude and latitude
            y = float(table_value[1])
            pnt = arcpy.PointGeometry(arcpy.Point(x,y),sr_gcs)
            pnt_alberse = pnt.projectAs(sr_albsers)

            row = (pnt_alberse,) + tuple(table_value)
            cursor.insertRow(row)

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



New point feature class 'point_albers' created in 'D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\GIS\Assignment_6\Assignment_6.gdb' with 5 fields.


Q1: Line by line, explain what was done in the code. (10 pnts)
•	Import modules: arcpy, csv, os.
•	Define paths for CSV, geodatabase, and output feature class.
•	Read CSV header to count fields (columns).
•	Set spatial references: 4269: NAD83 (geographic, lat/long) and 102039: NAD83/Conus Albers (projected, Albers Equal Area).
•	Create feature class path (fcName).
•	Delete existing feature class if present.
•	Create new point feature class.
•	Add missing CSV header fields as text fields to the feature class.
•	Use arcpy.da.InsertCursor to populate data: Read CSV rows, Convert lat/long to point geometry using 4269, Reproject to 102039, Insert geometry and CSV data into the feature class.
•	Print completion message.


Q2: What do the cod 4269 and 102309 represent? (5 pnts)
•	4269: EPSG code for NAD83 (latitude/longitude, geographic coordinate system).
•	102039: ESRI code for NAD83/Conus Albers (projected coordinate system, Albers Equal Area).


Q3: Explain what is SHAPE@ (5 pnts) 
•	SHAPE@: ArcPy cursor token for accessing/modifying a feature’s geometry.
•	Used to insert point/line/polygon shapes directly (e.g., coordinates in this script).
•	Essential for spatial data operations with cursors.


#Use arcpy.Exist() (10 pnts)
This code checks the existence of a specified dataset within an ArcGIS workspace.
Fill the dataset_name and workspace_path variables with the database and the feature class name (points) from the last block
Print a message indicating whether the dataset exists or not.

In [3]:
import arcpy
import os

# Prompt for user input
dataset_name = "point_albers"
workspace_path = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\GIS\Assignment_6\Assignment_6.gdb"
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

The dataset 'point_albers' exists in the workspace.


Q4: Line by line describe what was done by the code in the block above. (10 pnts)

•	import arcpy: Imports the ArcPy module for geospatial operations.
•	import os: Imports the OS module for file/path handling.
•	dataset_name = "point_albers": Defines the name of the dataset (feature class) to check.
•	workspace_path = r"D:\...\Assignment_6.gdb": Sets the path to the geodatabase workspace (raw string avoids escape issues).
•	fullname = os.path.join(workspace_path, dataset_name): Combines workspace path and dataset name into a full path (ensures OS compatibility).
•	if arcpy.Exists(fullname):: Checks if the dataset exists in the geodatabase.
1.	print(f"The dataset '{dataset_name}' exists..."): Outputs confirmation if the dataset is found.
2.	else:: Handles the case where the dataset is missing.
3.	print(f"The dataset '{dataset_name}' does not exist..."): Outputs a warning if the dataset is not found



#Use arcpy.Walk() (10 pnts)
This code uses arcpy.Walk() to iterate through all feature datasets in the geodatabase you just used.
Modify the name of "workspace" to make the code work
Run the code to list all the feature classes within each dataset.

In [4]:

import arcpy

# Define the workspace
workspace = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\GIS\Assignment_6\Assignment_6.gdb" # note that you need to put a full path name for this workspace, even running it in ArcGIS Pro. 

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")
     

Feature Class in D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\GIS\Assignment_6\Assignment_6.gdb: point_albers


In [ ]:
Q5: line by line, describe what was done by the code in the block above. (10 pnts)

•	Import arcpy – Loads the ArcPy module for GIS operations.
•	Define workspace – Sets the path to the geodatabase.
•	Use arcpy.da.Walk() – Iterates through the geodatabase to find feature classes.
•	Loop through filenames – Extracts each feature class name.
•	Print feature class details – Displays the directory and feature class name.


#Use List Comprehension (10 pnts)
The following block uses list comprehension to generate a list of .shp files in a specified folder.

Choose the folder name from assignment 4 data/paris subfolder where many shapefiles are located and use it for folder_path

Run the code to print the list of file names

In the second block, the code combines the os.walk() function with the list comprehension to list all shapefiles in a folder including subfolders

In [5]:
import os

# Specify the folder path
folder_path = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\ModelBuilder_data\ParisData"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)
     

Shapefiles in the folder:
Metro_Entrances.shp
Warehouses.shp


In [6]:

import os
# Specify the folder path
folder_path = r"D:\LSU_Spring 2025\GEOG 4057\Class Projects\Assignment 6\ModelBuilder_data\ParisData"

print("Shapefiles in the folder:")
for root, dirs, files in os.walk(folder_path):
    shp_files = [file for file in files if file.endswith(".shp")]
    for shp_file in shp_files:
            print(shp_file)

Shapefiles in the folder:
Metro_Entrances.shp
Warehouses.shp
Stores.shp
Parks.shp
Metro_Lines.shp
Metro_LinesAOI.shp
Metro_Stations.shp
Streets.shp


Q6: Description of the first code block (5 pts)

•	The folder path is defined to specify the directory containing the files.
•	List comprehension is used to filter and store only the files ending with the .shp extension in the shp_files variable.
•	A for loop iterates through shp_files, printing each shapefile name.– Displays the directory and feature class name.


Q7: Description of the second code block (5 pts)
•	os.walk() is used instead of os.listdir(), allowing recursive traversal through directories.
•	The variables root, dirs, and files represent the current folder, subdirectories, and files.
•	List comprehension is applied to filter out files that end with .shp, keeping only shapefiles.


In [7]:
import arcpy

# Prompt for user input
fc_path = geodatabase_path + "/" + feature_class_name
fields_to_extract = ["X", "Y", "Field1", "Field2", "Field3"]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract))])

Extracted Data:
['-118.2437', '34.0522', '10', '15', '20']
['-73.935242', '40.730610', '8', '11', '14']
['-95.369804', '29.760427', '12', '18', '22']
['-121.886329', '37.338208', '6', '9', '12']
['-87.629798', '41.878113', '15', '22', '28']
['-97.743061', '30.267153', '11', '16', '21']
['-122.419416', '37.774929', '9', '13', '17']
['-118.242766', '34.052235', '14', '21', '26']
['-71.058880', '42.360081', '7', '10', '13']
['-81.694360', '41.499321', '12', '17', '22']


Q8: Line-by-line explanation of the code (10 pts)
•	Import arcpy – Loads the ArcPy module to work with GIS data.
•	Define fc_path – Constructs the path to a feature class inside a geodatabase.
•	Specify fields to extract – Lists the attribute table fields to retrieve from the feature class.
•	Create a search cursor (arcpy.da.SearchCursor) – Iterates through each row of the feature class, extracting the specified fields.
•	Use with statement – Ensures the cursor closes automatically after execution.
•	Print extracted data – Displays the retrieved attribute values.


In [15]:

import arcpy
fc = "point_fromxy"
newfieldName = "all"
arcpy.AddField_management(fc, newfieldName, "DOUBLE")
expression = "sum(!Field1!,!Field2!,!Field3!)"
codeblock = """
def sum(*fields):
    sum = 0
    for field in fields:
        sum += field
    return sum
"""
arcpy.CalculateField_management(fc, newfieldName, expression, "", codeblock)
     

<Result 'D:\\LSU_Spring 2025\\GEOG 4057\\Class Projects\\Assignment 6\\Output_points\\point_fromxy.shp'>

Q9: Line-by-line explanation of the code (10 pts)
•	Import arcpy – Loads the ArcPy module for GIS processing.
•	Define fc (Feature Class) – Specifies the name of the feature class (point_fromxy).
•	Create a new field (AddField_management) – Adds a new field named "all" with a data type of DOUBLE to store numeric values.
•	Define expression – Uses the sum function to add values from Field1, Field2, and Field3.
•	Write a Python function (codeblock) – Defines a sum function that loops through the given fields and returns their sum.
•	Apply field calculation (CalculateField_management) – Computes the sum of the specified fields and updates the "all" field in the attribute table.


10: Field Calculator Interface & Comparison (10 pts)
Opening Attribute Table: The "all" field should be present with summed values from Field1, Field2, and Field3.
Field Calculator Interface:
1.	Supports VB Script and Python expressions.
2.	Allows manual input or built-in functions.
3.	Provides a preview for calculations.
Comparison:
1.	Python Code automates the process for repeatability.
2.	Field Calculator is a manual, GUI-based method.
3.	Python allows loops and reusability, while the Field Calculator is more limited.

